# 03: Add datasets

In this script, we add some custom datasets: Synfuel production with VRE inputs

In [1]:
%run common_definitions.py

In [2]:
import brightway2 as bw

In [3]:
bw.projects.set_current(BW_PROJECTNAME)

Electricity production datasets that make up the VRE inputs of the new synfuel production datasets.

In [4]:
VRE_ACTIVITIES = [
    "electricity production, wind, 1-3MW turbine, offshore",
    "electricity production, wind, 1-3MW turbine, onshore",
    "electricity production, wind, <1MW turbine, onshore",
    "electricity production, wind, >3MW turbine, onshore",
    "electricity production, photovoltaic, commercial"
]

In [5]:
def trim_electricity_inputs(act, vre_activities):
    # calculate VRE share
    vre_share = 0
    for exc in act.technosphere():
        if exc.input != act:
            if exc.input["name"] in vre_activities:
                vre_share += exc.amount

    # scale inputs, delete non-VRE inputs
    for exc in act.technosphere():
        if exc.input != act:
            if exc.input["name"] in vre_activities:
                exc["amount"] = exc["amount"] / vre_share
                exc.save()
            else:
                exc.delete()

def create_new_market_groups(region, db):
    # search old markets
    voltages = ["low", "medium", "high"]
    grid_markets = {}
    for v in voltages:
        name = "market group for electricity, {} voltage".format(v)
        matches = [act for act in db if act["name"]==name and act["location"]==region]
        if len(matches) != 1:
            raise ValueError("Not exactly one market found.")
        else:
            grid_markets[v] = matches[0]

    # copy them
    VRE_markets = {}
    for v in voltages:
        VRE_markets[v] = grid_markets[v].copy()

    # rename
    for v in voltages:
        act = VRE_markets[v]
        oldname = grid_markets[v]["name"]
        act["name"] = oldname + ", VRE inputs"
        act.save()

    # in the high voltage market, trim inputs
    trim_electricity_inputs(VRE_markets["high"], VRE_ACTIVITIES)

    # relink high to medium
    for exc in VRE_markets["medium"].exchanges():
        if exc.input == grid_markets["high"]:
            exc["input"] = VRE_markets["high"]
            exc.save()

    # relink medium to low
    for exc in VRE_markets["low"].exchanges():
        if exc.input == grid_markets["medium"]:
            exc["input"] = VRE_markets["medium"]
            exc.save()
            
    return VRE_markets


def create_feedstock_activities(region, new_elec_input, db):
    #green hydrogen production
    name = "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity"
    matches = [act for act in db if act["name"]==name and act["location"]==region]
    grid_h2 = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_h2 = matches[0]

    vre_h2 = grid_h2.copy()
    vre_h2["name"] = "hydrogen production, gaseous, 30 bar, from PEM electrolysis, VRE inputs"
    vre_h2.save()

    old_elec_input = "market group for electricity, low voltage"
    for exc in vre_h2.exchanges():
        if exc.input["name"] == old_elec_input:
            exc["input"] = new_elec_input["low"]
            exc.save()

    # CO2 capture
    name = "carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with heat pump heat, and grid electricity"
    matches = [act for act in db if act["name"]==name and act["location"]==region]
    grid_co2 = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_co2 = matches[0]

    vre_co2 = grid_co2.copy()
    vre_co2["name"] = name.replace("grid electricity", "VRE inputs")
    vre_co2.save()

    old_elec_input = "market group for electricity, medium voltage"
    for exc in vre_co2.exchanges():
        if exc.input["name"] == old_elec_input:
            exc["input"] = new_elec_input["medium"]
            exc.save()

    # CO production
    name = "carbon monoxide, from RWGS, for Fischer Tropsch process, hydrogen from electrolysis"
    matches = [act for act in db if act["name"]==name and act["location"]==region]
    grid_co = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_co = matches[0]

    vre_co = grid_co.copy()
    vre_co["name"] = "carbon monoxide, from RWGS, for Fischer Tropsch process, hydrogen from electrolysis, VRE inputs"
    vre_co.save()

    for exc in vre_co.exchanges():
        if exc.input == grid_h2:
            exc["input"] = vre_h2
            exc.save()
        if exc.input == grid_co2:
            exc["input"] = vre_co2
            exc.save()

    # syngas production
    name = "syngas, RWGS, Production, for Fischer Tropsch process, hydrogen from electrolysis"
    matches = [act for act in db if act["name"]==name and act["location"]==region]
    grid_syngas = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_syngas = matches[0]

    vre_syngas = grid_syngas.copy()
    vre_syngas["name"] = "syngas, RWGS, Production, for Fischer Tropsch process, hydrogen from electrolysis, VRE inputs"
    vre_syngas.save()

    for exc in vre_syngas.exchanges():
        if exc.input == grid_h2:
            exc["input"] = vre_h2
            exc.save()
        if exc.input == grid_co:
            exc["input"] = vre_co
            exc.save()

    activities = {
        "h2": vre_h2,
        "co2": vre_co2,
        "co": vre_co,
        "syngas": vre_syngas
    }

    return activities


def create_fuel_activites(region, fuelname, new_syngas_input, db):
    matches = [act for act in db if act["name"]==fuelname and act["location"]==region]
    grid_fuel = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_fuel = matches[0]

    vre_fuel = grid_fuel.copy()
    newname = fuelname.replace("hydrogen from electrolysis", "hydrogen from electrolysis, VRE inputs")
    vre_fuel["name"] = newname
    vre_fuel.save()

    old_syngas_input = "syngas, RWGS, Production, for Fischer Tropsch process, hydrogen from electrolysis"
    for exc in vre_fuel.exchanges():
        if exc.input["name"] == old_syngas_input:
            exc["input"] = new_syngas_input
            exc.save()

    return vre_fuel

def create_storage_activities(region, new_h2_input, db):
    ei_name = "hydrogen storage, for grid-balancing"
    matches = [act for act in db if act["name"]==ei_name and act["location"]==region]
    grid_storage = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_storage = matches[0]

    vre_storage = grid_storage.copy()
    newname = ei_name + ", VRE inputs"
    vre_storage["name"] = newname
    vre_storage.save()

    old_h2_input = "hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity"
    for exc in vre_storage.exchanges():
        if exc.input["name"] == old_h2_input:
            exc["input"] = new_h2_input
            exc.save()

    return vre_storage

def create_h2turbine_activities(region, new_storage_input, db):
    ei_name = "electricity production, from hydrogen-fired one gigawatt gas turbine"
    matches = [act for act in db if act["name"]==ei_name and act["location"]==region]
    grid_turbine = None
    if len(matches) != 1:
        raise ValueError("Not exactly one activity found.")
    else:
        grid_turbine = matches[0]

    vre_turbine = grid_turbine.copy()
    newname = ei_name + ", VRE inputs"
    vre_turbine["name"] = newname
    vre_turbine.save()

    old_storage_input = "hydrogen storage, for grid-balancing"
    for exc in vre_turbine.exchanges():
        if exc.input["name"] == old_storage_input:
            exc["input"] = new_storage_input
            exc.save()

    return vre_turbine

def clean_database(db):
    for act in db:
        if "VRE inputs" in act["name"]:
            print("Removing {}.".format(str(act)))
            act.delete()

## Get list of pLCA databases

In [6]:
dblist = [db for db in bw.databases if "remind" in db]
dblist

["ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-NPi_2030_2024-05-06 14-54 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-NPi_2040_2024-05-06 15-06 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-NPi_2050_2024-05-06 15-15 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2020_2024-05-06 15-25 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2030_2024-05-06 15-38 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2040_2024-05-06 15-53 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)",
 "ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050_2024-05-06 16-14 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)"]

In [7]:
for dbname in dblist:
    db = bw.Database(dbname)
    clean_database(db)

Removing 'syngas, RWGS, Production, for Fischer Tropsch process, hydrogen from electrolysis, VRE inputs' (kilogram, NEU, None).
Removing 'carbon monoxide, from RWGS, for Fischer Tropsch process, hydrogen from electrolysis, VRE inputs' (kilogram, REF, None).
Removing 'carbon monoxide, from RWGS, for Fischer Tropsch process, hydrogen from electrolysis, VRE inputs' (kilogram, LAM, None).
Removing 'carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with heat pump heat, and VRE inputs' (kilogram, LAM, None).
Removing 'carbon monoxide, from RWGS, for Fischer Tropsch process, hydrogen from electrolysis, VRE inputs' (kilogram, CHA, None).
Removing 'market group for electricity, medium voltage, VRE inputs' (kilowatt hour, USA, None).
Removing 'carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with heat pump heat, and VRE inputs' (kilogram, MEA, None).
Removing 'hydrogen production, gaseous, 30 bar, fr

## Create diesel production datasets

In [8]:
fuelname = "diesel production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation"

for dbname in dblist:
    print("Creating VRE activities for {}".format(dbname))

    db = bw.Database(dbname)
    matches = [act for act in db if act["name"] == fuelname]
    regions = [act["location"] for act in matches if act["location"] != "RER"]
    
    for i, region in enumerate(regions):
        mkts = create_new_market_groups(region, db)
        feedstocks = create_feedstock_activities(region, mkts, db)
        storage_act = create_storage_activities(region, feedstocks["h2"], db)
        fuel_act = create_fuel_activites(region, fuelname, feedstocks["syngas"], db)
        turbine_act = create_h2turbine_activities(region, storage_act, db)
        print("\t{} of {} regions done.".format(i+1, len(regions)))

Creating VRE activities for ecoinvent_cutoff_3.9_remind_SSP2-NPi_2020_2024-05-06 14-43 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	1 of 13 regions done.
	2 of 13 regions done.
	3 of 13 regions done.
	4 of 13 regions done.
	5 of 13 regions done.
	6 of 13 regions done.
	7 of 13 regions done.
	8 of 13 regions done.
	9 of 13 regions done.
	10 of 13 regions done.
	11 of 13 regions done.
	12 of 13 regions done.
	13 of 13 regions done.
Creating VRE activities for ecoinvent_cutoff_3.9_remind_SSP2-NPi_2030_2024-05-06 14-54 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	1 of 13 regions done.
	2 of 13 regions done.
	3 of 13 regions done.
	4 of 13 regions done.
	5 of 13 regions done.
	6 of 13 regions done.
	7 of 13 regions done.
	8 of 13 regions done.
	9 of 13 regions done.
	10 of 13 regions done.
	11 of 13 regions done.
	12 of 13 regions done.
	13 of 13 regions done.
Creating VRE activities for ecoinvent_cutoff_3.9_remind_SSP2-NPi_2040_2024-05-06 15-06 v.(.2.,. .1.,. .0.,. .'.d.e.v.5.'.)
	1 of 13 regions done.
	2